- - - - - - - - - - - - - - - - - - -
# 1. Imports
## Relevent Modules - purpose:
    - Pandas     - dataframe, data representation
    - Numpy      - data manipulation, random generation
    - Sklearn    - Regression models, data split, one-hot encoding, 
                   metrics, cross validation
    - Matplotlib - Data visualization
    - Seaborn    - Data visualization
    - Random     - Shuffling data

## Module Versions:
    - Pandas:     '1.5.3'
    - Numpy:      '1.24.3'
    - Sklearn:    '1.3.0'
    - Matplotlib: '3.7.1'
    - Seaborn:    '0.12.2'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from random import shuffle

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import seaborn as sb

- - - - - - - - - - - - - - - - - - -
# 2. Data
 - Reads in the data from a cleaned file
 - Removes null lines and corrects empty class for 'fel_misd'
 - Normalizes the relevent columns
 - Remove the oversized and generate data for undersized classes:
     - M: 46803 $\to$ 2194
     - F: 16407 $\to$ 2194
     - C: 2194  $\to$ 2194
     - S: 240   $\to$  360
     - P: 50    $\to$   75
     
     - Note that the data generation is done using the distribution
       of each of the inputs per each class. Since the inputs have
       low correleation (see the correlation heatmap) the joint pdf
       will be extremely close to the individual pdf for each input.
 - One hot encode the inputs

---
## 2.1 Reading Data

In [ ]:
df = pd.read_csv("./clean_data/fully_merged_data.csv")
df.shape

In [ ]:
# delete empty values
df = df.dropna()
df.shape

In [ ]:
# remove null values
arr = df.index[df["fel_misd"] == ' ']
df = df.drop(arr, axis=0)
arr = df.index[df["fel_misd"] == '\xa0']
df = df.drop(arr, axis=0)
df.shape

---
## 2.2 Normalizing data

In [ ]:
# z-score normalize desired columns
from scipy.stats import zscore

need_norm = ["age","MEDHINC_CY", "WLTHINDXCY", "TOTHH_CY"]
norm = df[need_norm].apply(zscore)
norm.head()

In [ ]:
df[need_norm] = norm
df.head()

---
## 2.3 Over and under-sized classes

In [ ]:
df["fel_misd"].value_counts()

---
### 2.3.1 Drop random points from oversized data

In [ ]:
m_arr = df.index[df["fel_misd"] == 'M'].tolist()
shuffle(m_arr)


df = df.drop(m_arr[0:len(m_arr)-2194], axis = 0)


s_arr = df.index[df["fel_misd"] == 'S'].tolist()
p_arr = df.index[df["fel_misd"] == 'P'].tolist()

f_arr = df.index[df["fel_misd"] == 'F'].tolist()
shuffle(f_arr)
df = df.drop(f_arr[0:len(f_arr)-2194], axis = 0)

df['fel_misd'].value_counts()

---
### 2.3.2 Statistical Generation for Undersized Classes

In [ ]:
def gen_rand_df(temp_df,samples=1):
    ''' 
    Assume that temp_df is only populated with same fel_misd class and no one-hot 
    encoding
    Age, MEDHINC_CY, WLTHINDXCY, time_arr, TOTHH_CY should be normalized prior
    to calling this function 
    '''

    # Dictionary to put into data frame
    d = {}
    
    # Find the pdf for the 'sex' input
    choices = temp_df['sex'].value_counts().index.to_list()
    v_c = temp_df['sex'].value_counts()
    probs = v_c/sum(v_c)
    
    # Update the dictionary at 'sex' to data generated from pdf
    d['sex'] = np.random.choice(choices, p=probs, size=samples)
    
    # Repeat...
    choices = temp_df['day'].value_counts().index.to_list()
    v_c = temp_df['day'].value_counts()
    probs = v_c/sum(v_c)    
    d['day'] = np.random.choice(choices, p=probs,size=samples)
    
    choices = temp_df['month'].value_counts().index.to_list()
    v_c = temp_df['month'].value_counts()
    probs = v_c/sum(v_c)    
    d['month'] = np.random.choice(choices, p=probs, size=samples)
    
    x = np.random.normal(0,1,size=(5,samples))
    d['age'] = x[0]
    d['MEDHINC_CY'] = x[1]
    d['WLTHINDXCY'] = x[2]
    d['time_arr'] = x[3]
    d['TOTHH_CY'] = x[4]
    d['fel_misd'] = [temp_df['fel_misd'].to_list()[0] for i in range(samples)]
    
    df_return = pd.DataFrame.from_dict(d)

    
    return df_return

# Increase the undersized class by 50%
s_amt = (int) (0.5*240)  
p_amt = (int) (0.5*50)    
inp = df[df['fel_misd'] == 'S']
s_temp = gen_rand_df(inp,s_amt)

inp = df[df['fel_misd'] == 'P']
p_temp = gen_rand_df(inp,p_amt)

df = pd.concat([df,s_temp,p_temp])
df['fel_misd'].value_counts()

---
## 2.4 One-Hot Encoding

In [ ]:
df_pandas_encoded = pd.get_dummies(df, columns=['sex', 'day', 'month'], drop_first=True)
df_pandas_encoded = df_pandas_encoded.drop("WLTHINDXCY", axis=1)
df_pandas_encoded.head()

- - - - - - - - - - - - - - - - - - -
# 3. Pre-Training
- Find the train and test sets (0.8 train split)
- Find all the combinations of the inputs
- Perform 5-fold cross validation for each combination

---
## 3.1 Find the Input and Output names

In [ ]:
inp = list(df_pandas_encoded.columns)
oup = ["fel_misd"]
for x in oup:
    inp.remove(x)
inp, oup

---
## 3.2 Split Train and Test Data

In [ ]:
x,y = df_pandas_encoded[inp], df_pandas_encoded[oup]
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, test_size=0.2)

---
## 3.3 Find Combinations

In [ ]:
import itertools
items = ['MEDHINC_CY','age','sex_M', 'day', 'month']
combs = []
for i in range(1, len(items)):
    combs.append(list(set(itertools.combinations(items, i))))

In [ ]:
day_vals = ['day_1',
          'day_2',
          'day_3',
          'day_4',
          'day_5',
          'day_6']

month_vals = ['month_1',
              'month_2',
              'month_3',
              'month_4',
              'month_5',
              'month_6',
              'month_7',
              'month_8',
              'month_9',
              'month_10',
              'month_11']

best_dict = {"features": [], "score": -2**31}
for k_amt in combs:
    for ind_comb in k_amt:
        comb = list(ind_comb)
        if 'day' in comb:
            comb.remove('day')
            comb += day_vals
        if 'month' in comb:
            comb.remove('month')
            comb += month_vals

        
        x_subset = x_train[comb].values
        cvs = cross_val_score(linear_model.LogisticRegression(multi_class='ovr'), x_subset, y_train)
        
        if cvs.mean() > best_dict["score"]:
            best_dict["features"] = comb
            best_dict["score"] = cvs.mean()
best_dict

-------
# 4. Training Logistic Model
- Train a 5-class logistic regression using one-vs-rest classification
    - Use the best features calculated in cross validation
- Fit the train data
- Find the predicted data for the test set and show the accuracy
- Visualize and show the confusion matrix using heatmap
- Show the classification report
- Show the weights for each input and model
    - Both tabular and heatmap

---
## 4.1 Training

In [ ]:
log_r = linear_model.LogisticRegression(penalty='l2', multi_class="ovr")

In [ ]:
best_dict["features"]

In [ ]:
x_train = x_train[best_dict["features"]]
x_test = x_test[best_dict["features"]]

log_r.fit(x_train, y_train)

---
## 4.2 Analysis
---
### 4.2.1 Accuracy

In [ ]:
y_pred = log_r.predict(x_test)
print(f"Accuracy train: {log_r.score(x_train,y_train)}")
print(f"Accuracy Test: {log_r.score(x_test,y_test)}")

---
### 4.2.2 Confusion Matrix

In [ ]:
temp = np.arange(0.5,5,1)
cm = confusion_matrix(y_test, y_pred)
sb.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xticks(temp, labels=log_r.classes_)
plt.yticks(temp, labels=log_r.classes_)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title("Confusion Matrix Heatmap")
plt.show()

In [ ]:
cm

---
### 4.2.3 Classification Report

In [ ]:
print(classification_report(y_test, y_pred))

---
### 4.2.4 Coefficient Weights

In [ ]:
df = pd.DataFrame(log_r.coef_, columns=log_r.feature_names_in_)
df = df.set_index(log_r.classes_)
df

In [ ]:
sb.heatmap(df.T, annot=True, cmap='Blues')
plt.xlabel('Class')
plt.ylabel('Input')
plt.title("Heatmap of Coefficients for Logistic Regression")
plt.show()

---
# 5. Pre-Training
- Find the train and test sets (0.8 train split)
- Find all the combinations of the inputs
- Perform 5-fold cross validation for each combination
---
## 5.1 Find the Input and Output  Names

In [ ]:
inp = list(df_pandas_encoded.columns)
oup = ["MEDHINC_CY"]
for x in oup:
    inp.remove(x)
inp.remove('fel_misd')
inp, oup

---
## 5.2 Split Train and Test Data

In [ ]:
x,y = df_pandas_encoded[inp], df_pandas_encoded[oup]
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, test_size=0.2)

---
## 5.3 Find Combinations
---
### 5.3.1 5-Fold CV for Ridge Regression

In [ ]:
items = ['age','sex_M', 'day', 'month']
combs = []
for i in range(1, len(items)):
    combs.append(list(set(itertools.combinations(items, i))))

In [ ]:
day_vals = ['day_1',
          'day_2',
          'day_3',
          'day_4',
          'day_5',
          'day_6']

month_vals = ['month_1',
              'month_2',
              'month_3',
              'month_4',
              'month_5',
              'month_6',
              'month_7',
              'month_8',
              'month_9',
              'month_10',
              'month_11']

best_dict = {"features": [], "score": -2**31, "alpha": 0}
alpha_vals = np.logspace(-1,3,50)
for k_amt in combs:
    for ind_comb in k_amt:
        comb = list(ind_comb)
        if 'day' in comb:
            comb.remove('day')
            comb += day_vals
        if 'month' in comb:
            comb.remove('month')
            comb += month_vals

        
        x_subset = x_train[comb].values
        
        for alpha in alpha_vals:
            cvs = cross_val_score(linear_model.Ridge(alpha=alpha), x_subset, y_train)
        
        if cvs.mean() > best_dict["score"]:
            best_dict["features"] = comb
            best_dict["score"] = cvs.mean()
best_dict

---
### 5.3.2 20-Fold CV for Lasso Regression

In [ ]:
lasso = linear_model.LassoLarsCV(cv=20)
lasso.fit(x_train, y_train)
lasso_mse=lasso.mse_path_
lasso_alphas = lasso.cv_alphas_
mse_mean = []
mse_std = []
for i in range(len(lasso_mse)):
    mse_mean.append(lasso_mse[i].mean())
    mse_std.append(lasso_mse[i].std())
    
min_alpha = lasso_alphas[np.argmin(mse_mean)]
print(f"Best Average MSE: {min(mse_mean)} with {min_alpha=}")

In [ ]:
plt.errorbar(lasso_alphas,mse_mean, yerr=mse_std,fmt="o", capsize=5, color="blue")
plt.scatter(lasso_alphas, mse_mean, color='blue')
plt.xscale('log')

plt.axvline(x=min_alpha, color='red')
plt.xlabel('Alpha')
plt.ylabel('Mean MSE')
plt.title('Alpha vs Mean MSE');

-----
# 6. Training Regression Models
- Find the input variables wanted
- Split the data
- Find all the possible combinations of inputs
- Use cross validation for multiple alpha values for a Ridge model
- Use cross validation on the Lasso model
- Visualize the Lasso alphas
- Fit the Ridge regression (Linear regression since $\alpha=0$
- Show the relevent information
    - $R^2$ 
    - Residual plot
- Fit the Lasso regression with the best $\alpha$
- Show the relevent information
    - $R^2$ 
    - Residual plot
- Show the weights for each input and model
    - Both tabular and heatmap

---
## 6.1 Training Linear Regression

In [ ]:
# Ridge alpha is 0, use linear regression: https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.Ridge.html
# Lasso alpha is 0.0067 try lasso
lr = linear_model.LinearRegression()

x_train = x_train[best_dict["features"]]
x_test = x_test[best_dict["features"]]

lr.fit(x_train, y_train)

---
## 6.2 Analysis
---
### 6.2.1 Coefficient of Determination: $R^2$ 

In [ ]:
y_pred = lr.predict(x_test)
print(f"R2 train: {lr.score(x_train,y_train)}")
print(f"R2 Test: {lr.score(x_test,y_test)}")

---
### 6.2.2 Residual Plot 

In [ ]:
plt.scatter([i for i in range(len(y_test))],y_test-y_pred);
plt.xlabel("Index")
plt.ylabel("y_actual-y_predicted")
plt.title("Residual Plot");

---
## 6.3 Training Lasso Regression

In [ ]:
lr_lasso = linear_model.Lasso(alpha=min_alpha)
lr_lasso.fit(x_train, y_train)

---
## 6.4 Analysis
---
### 6.4.1 Coefficient of Determination: $R^2$

In [ ]:
y_lasso_pred = lr_lasso.predict(x_test)
print(f"R2 train: {lr_lasso.score(x_train,y_train)}")
print(f"R2 Test: {lr_lasso.score(x_test,y_test)}")

---
### 6.4.2 Residual Plot

In [ ]:
plt.scatter([i for i in range(len(y_test))],y_test.values.reshape(-1)-y_lasso_pred);
plt.xlabel("Index")
plt.ylabel("y_actual-y_predicted")
plt.title("Residual Plot");

---
## 6.5 Coefficient Weights

In [ ]:
# Insert code here
df1 = pd.DataFrame(lr.coef_, columns=lr.feature_names_in_)
df2 = pd.DataFrame(lr_lasso.coef_.reshape(1,-1), columns=lr_lasso.feature_names_in_)
df = pd.concat([df1,df2])
df["Name"] = ["Linear Regression", "Lasso"]
df = df.set_index(df["Name"])
df = df.drop("Name", axis=1)
df

In [ ]:
sb.heatmap(df.T, annot=True, cmap='Blues')
plt.xlabel('Type of Model')
plt.ylabel('Input')
plt.title("Heatmap of Coefficients for Inputs per Model")
plt.show()